In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import lightkurve as lk
from astropy.timeseries import LombScargle

from phi_m_alg import phi_m_radio as pmr

%matplotlib widget

# getting kepler ID data: kic7917485

In [ ]:
target_kepler_id = 'kic7917485'
instrument = 'kepler'
cadence = 'long'
author='kepler'

lc_files = lk.search_lightcurve(target_kepler_id, cadence=cadence,author=author).download_all()
        
time, flux, flerr = np.array([]), np.array([]), np.array([])

for q,lc in enumerate(lc_files):
    lc = lc.remove_outliers(sigma=4)
    this_time = np.ma.array(lc['time'].value)

    ### to get rid of the masks ###
    this_flux = lc['pdcsap_flux'].value
    this_flerr = lc['pdcsap_flux_err'].value

    good = np.isfinite(this_time) & np.isfinite(this_flux) & np.isfinite(this_flerr)
    
    median_flux = np.nanmedian(this_flux)
    this_flux = this_flux[good] / median_flux
    this_flerr = this_flerr[good] / median_flux
    this_time = this_time[good]
    
    time = np.concatenate((time,this_time))
    flux = np.concatenate((flux,this_flux))
    flerr = np.concatenate((flerr,this_flerr))

flux=flux-1.

i = np.argsort(time)

ts, ys, flerrs = time[i],flux[i],flerr[i]

In [ ]:
# for finding the strongest mode
mode_f = pmr.tune(ts,ys)

In [ ]:
### running phi-m radio method ###
qs = pmr.demodulate(ts,ys,mode_f)
fs, ps = pmr.listen(ts,qs,mode_f)

In [ ]:
plt.figure()
plt.plot(1/fs,ps,color='black')
plt.loglog()
plt.axvline(840)